<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Acknowledgements" data-toc-modified-id="Acknowledgements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Acknowledgements</a></span></li><li><span><a href="#Prepare-data-and-model" data-toc-modified-id="Prepare-data-and-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare data and model</a></span></li><li><span><a href="#Make-feature-matrix-(word2vec,-votes,-stars)" data-toc-modified-id="Make-feature-matrix-(word2vec,-votes,-stars)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make feature matrix (word2vec, votes, stars)</a></span></li><li><span><a href="#Create-Label-y-(Business-categories)" data-toc-modified-id="Create-Label-y-(Business-categories)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Label y (Business categories)</a></span></li><li><span><a href="#Join-x,y-(feature-matrix,-category)-using-business_id" data-toc-modified-id="Join-x,y-(feature-matrix,-category)-using-business_id-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Join x,y (feature matrix, category) using business_id</a></span></li><li><span><a href="#Category-Prediction" data-toc-modified-id="Category-Prediction-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Category Prediction</a></span><ul class="toc-item"><li><span><a href="#Plot-ROC-curve-to-assess" data-toc-modified-id="Plot-ROC-curve-to-assess-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Plot ROC curve to assess</a></span></li></ul></li><li><span><a href="#Cluster-with-metadata-(useful,-cool,-funny,-stars)" data-toc-modified-id="Cluster-with-metadata-(useful,-cool,-funny,-stars)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Cluster with metadata (useful, cool, funny, stars)</a></span></li></ul></div>

# Acknowledgements
Thanks to the tutorial: https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-3-more-fun-with-word-vectors

# Prepare data and model

In [92]:
%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt

import re

import nltk
import nltk.data
nltk.download('stopwords')
from nltk.corpus import stopwords # Import the stop word list



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daviderickson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def load_reviews(size='small'): 
    if size == 'small':
        filename = r'../../data/small-review.json'
    elif size == 'intermediate':
        filename = r'../../data/intermediate-review.json'
    elif size == 'full':
        filename = r'../../data/review.json'
    new_list = []
    for line in open(filename):
       new_list.append(json.loads(line))
    return pd.DataFrame.from_records(new_list)

dfreviews = load_reviews(size='intermediate')

In [3]:
dfreviews.head()

business_id  cool                 date  funny  \
0  ujmEBvifdJM6h6RLv4wQIg     0  2013-05-07 04:34:36      1   
1  NZnhc2sEQy3RmzKTZnqtwQ     0  2017-01-14 21:30:33      0   
2  WTqjgwHlXbSFevF32_DJVw     0  2016-11-09 20:09:03      0   
3  ikCg8xy5JIg_NGPx-MSIDA     0  2018-01-09 20:56:38      0   
4  b1b1eb3uo-w561D0ZfCEiQ     0  2018-01-30 23:07:38      0   

                review_id  stars  \
0  Q1sbwvVQXV2734tPgoKj4Q    1.0   
1  GJXCdrto3ASJOqKeVWPi6Q    5.0   
2  2TzJjDVDEuAW6MR5Vuc1ug    5.0   
3  yi0R0Ugj_xUx_Nek0-_Qig    5.0   
4  11a8sVPMUFtaC7_ABRkmtw    1.0   

                                                text  useful  \
0  Total bill for this horrible service? Over $8G...       6   
1  I *adore* Travis at the Hard Rock's new Kelly ...       0   
2  I have to say that this office really has it t...       3   
3  Went in for a lunch. Steak sandwich was delici...       0   
4  Today was my second out of three sessions I ha...       7   

                  user_id  
0  hG7b0MtEbXx5QzbzE6C_VA  
1  yXQM5uF2jS6es16SJzNHfg  
2  n6-Gk65cPZL6Uz8qRm3NYw  
3  dacAIZ6fTM6mqwW5uxkskg  
4  ssoyf2_x0EQMed6fgHeMyQ

In [4]:
dfreviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [5]:
dfreviews['text'][0]

'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.'

In [6]:
# For simplicity, drop anything that isn't a letter
# Numbers and symbols may have interesting meaning and could be explore later

def lettersOnly(string):
    return re.sub("[^a-zA-Z]", " ", string) 

dfreviews['text'] = dfreviews['text'].apply(lettersOnly)


In [7]:
dfreviews['text'][0]

'Total bill for this horrible service  Over   Gs  These crooks actually had the nerve to charge us     for   pills  I checked online the pills can be had for    cents EACH  Avoid Hospital ERs at all costs '

In [8]:
def review_to_wordlist(string, remove_stopwords=False):
    string = re.sub("[^a-zA-Z]", " ", string) # keep only letters. more complex model possible later
    words =  string.lower().split() # make everything lowercase. split into words
    if remove_stopwords:
        stops = set(stopwords.words('english')) # create a fast lookup for stopwords
        words = [w for w in words if not w in stops] # remove stopwords
    return( words) # return a list of words
    
# dfreviews['text'] = dfreviews['text'].apply(review_to_words) # apply to reviews in dataframe


In [9]:
# Word2Vec expects single sentences, each one as a list of words

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [10]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences")
for review in dfreviews["text"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences


In [11]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2020-01-17 17:29:12,471 : INFO : 'pattern' package not found; tag filters are not available for English
2020-01-17 17:29:12,481 : INFO : collecting all words and their counts
2020-01-17 17:29:12,482 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-01-17 17:29:12,667 : INFO : PROGRESS: at sentence #10000, processed 1088334 words, keeping 25539 word types


Training model...


2020-01-17 17:29:12,846 : INFO : PROGRESS: at sentence #20000, processed 2172597 words, keeping 35463 word types
2020-01-17 17:29:13,011 : INFO : PROGRESS: at sentence #30000, processed 3251616 words, keeping 42649 word types
2020-01-17 17:29:13,188 : INFO : PROGRESS: at sentence #40000, processed 4373996 words, keeping 48893 word types
2020-01-17 17:29:13,358 : INFO : PROGRESS: at sentence #50000, processed 5471587 words, keeping 53964 word types
2020-01-17 17:29:13,527 : INFO : PROGRESS: at sentence #60000, processed 6570064 words, keeping 58362 word types
2020-01-17 17:29:13,696 : INFO : PROGRESS: at sentence #70000, processed 7667364 words, keeping 62704 word types
2020-01-17 17:29:13,866 : INFO : PROGRESS: at sentence #80000, processed 8768955 words, keeping 66443 word types
2020-01-17 17:29:14,035 : INFO : PROGRESS: at sentence #90000, processed 9872097 words, keeping 70199 word types
2020-01-17 17:29:14,208 : INFO : collected 73717 word types from a corpus of 10978770 raw words 

2020-01-17 17:29:56,274 : INFO : EPOCH 5 - PROGRESS: at 42.21% examples, 810529 words/s, in_qsize 7, out_qsize 0
2020-01-17 17:29:57,284 : INFO : EPOCH 5 - PROGRESS: at 53.21% examples, 817122 words/s, in_qsize 7, out_qsize 0
2020-01-17 17:29:58,302 : INFO : EPOCH 5 - PROGRESS: at 63.46% examples, 813511 words/s, in_qsize 7, out_qsize 0
2020-01-17 17:29:59,314 : INFO : EPOCH 5 - PROGRESS: at 71.62% examples, 787696 words/s, in_qsize 7, out_qsize 0
2020-01-17 17:30:00,314 : INFO : EPOCH 5 - PROGRESS: at 80.73% examples, 777280 words/s, in_qsize 7, out_qsize 0
2020-01-17 17:30:01,319 : INFO : EPOCH 5 - PROGRESS: at 91.63% examples, 785717 words/s, in_qsize 7, out_qsize 0
2020-01-17 17:30:02,111 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-01-17 17:30:02,120 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-01-17 17:30:02,121 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-01-17 17:30:02,143 : INFO : worker thread fi

In [12]:
model.most_similar('pizza')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('crust', 0.7031250596046448),
 ('pizzas', 0.6815289855003357),
 ('pepperoni', 0.675153374671936),
 ('margherita', 0.6247320771217346),
 ('calzone', 0.6213208436965942),
 ('lasagna', 0.5487133860588074),
 ('mozzarella', 0.5260012149810791),
 ('pasta', 0.5250520706176758),
 ('dough', 0.521996021270752),
 ('subs', 0.5141575336456299)]

In [13]:
model.most_similar('service')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('waitstaff', 0.5392504930496216),
 ('staff', 0.4584592878818512),
 ('hostesses', 0.4309860169887543),
 ('hospitality', 0.4167857766151428),
 ('communication', 0.4150824546813965),
 ('bartenders', 0.4137731194496155),
 ('servers', 0.40130314230918884),
 ('value', 0.3976908028125763),
 ('food', 0.3945622742176056),
 ('vibes', 0.3835543394088745)]

In [14]:
model.most_similar('bad')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.6269491910934448),
 ('horrible', 0.5781732797622681),
 ('good', 0.5464430451393127),
 ('poor', 0.5298254489898682),
 ('awful', 0.517661988735199),
 ('disappointing', 0.49108368158340454),
 ('alright', 0.4890827238559723),
 ('subpar', 0.46436044573783875),
 ('greatest', 0.4499537944793701),
 ('acceptable', 0.4481072425842285)]

In [15]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # WV.Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = int(0.)
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [16]:
# ****************************************************************
# Calculate average feature vectors
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_reviews = []
for review in dfreviews["text"]:
    clean_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

reviewDataVecs = getAvgFeatureVecs( clean_reviews, model, num_features )

Review 0 of 100000


/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 100000
Review 2000 of 100000
Review 3000 of 100000
Review 4000 of 100000
Review 5000 of 100000
Review 6000 of 100000
Review 7000 of 100000
Review 8000 of 100000


/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


Review 9000 of 100000
Review 10000 of 100000
Review 11000 of 100000
Review 12000 of 100000
Review 13000 of 100000
Review 14000 of 100000
Review 15000 of 100000
Review 16000 of 100000
Review 17000 of 100000
Review 18000 of 100000
Review 19000 of 100000
Review 20000 of 100000
Review 21000 of 100000
Review 22000 of 100000
Review 23000 of 100000
Review 24000 of 100000
Review 25000 of 100000
Review 26000 of 100000
Review 27000 of 100000
Review 28000 of 100000
Review 29000 of 100000
Review 30000 of 100000
Review 31000 of 100000
Review 32000 of 100000
Review 33000 of 100000
Review 34000 of 100000
Review 35000 of 100000
Review 36000 of 100000
Review 37000 of 100000
Review 38000 of 100000
Review 39000 of 100000
Review 40000 of 100000
Review 41000 of 100000
Review 42000 of 100000
Review 43000 of 100000
Review 44000 of 100000
Review 45000 of 100000
Review 46000 of 100000
Review 47000 of 100000
Review 48000 of 100000
Review 49000 of 100000
Review 50000 of 100000
Review 51000 of 100000
Review 52000

# Make feature matrix (word2vec, votes, stars)

In [17]:
reviewDataVecs.shape[1]

300

In [18]:
# Add non-text data back to feature matrix
review_features = ['cool', 'funny', 'useful', 'stars' , 'business_id']
all_features_labels = ['w2v{}'.format(idx) for idx in range(reviewDataVecs.shape[1])] + review_features
all_features = np.append(reviewDataVecs, dfreviews[review_features].to_numpy(), 1)


In [19]:
# Create df 
all_features_df = pd.DataFrame(data=all_features, columns=all_features_labels)

# Convert all but business_id to numerical
business_ids = all_features_df['business_id']
all_features_df = all_features_df.iloc[:,:-1].astype('float64')
all_features_df['business_id'] = business_ids
del business_ids

# Group by business_id
all_features_business = all_features_df.groupby(by='business_id').mean()

In [20]:
all_features_business.head()

w2v0      w2v1      w2v2      w2v3      w2v4  \
business_id                                                                
--I7YYLada0tSLkORTHb5Q  0.016423  0.028373 -0.017573  0.015987 -0.000105   
--U98MNlDym2cLn36BBPgQ  0.009526  0.020788 -0.013227  0.037498  0.006747   
--j-kaNMCo1-DYzddCsA5Q  0.035586  0.022514 -0.006759 -0.034605 -0.004300   
--wIGbLEhlpl_UeAIyDmZQ -0.006451  0.005224 -0.009912  0.012220 -0.001507   
-000aQFeK6tqVLndf7xORg -0.007433 -0.000011 -0.009821  0.013711 -0.007697   

                            w2v5      w2v6      w2v7      w2v8      w2v9  \
business_id                                                                
--I7YYLada0tSLkORTHb5Q -0.000613  0.022004 -0.017549 -0.007508 -0.007643   
--U98MNlDym2cLn36BBPgQ -0.004874  0.012273 -0.024117  0.008519 -0.009936   
--j-kaNMCo1-DYzddCsA5Q -0.001136  0.001259 -0.030018 -0.024447 -0.019531   
--wIGbLEhlpl_UeAIyDmZQ  0.014311 -0.004426 -0.013908 -0.003054 -0.003489   
-000aQFeK6tqVLndf7xORg  0.009369 -0.000267 -0.014385  0.013446 -0.006545   

                           w2v10     w2v11     w2v12     w2v13     w2v14  \
business_id                                                                
--I7YYLada0tSLkORTHb5Q -0.004137  0.005752  0.021792  0.008580 -0.005153   
--U98MNlDym2cLn36BBPgQ -0.003010 -0.007703  0.018541  0.022424 -0.008550   
--j-kaNMCo1-DYzddCsA5Q  0.013469 -0.015355  0.014754 -0.012143  0.011463   
--wIGbLEhlpl_UeAIyDmZQ -0.005158 -0.013398 -0.009127 -0.013979 -0.009750   
-000aQFeK6tqVLndf7xORg -0.004254 -0.008840  0.006065 -0.020545 -0.014723   

                           w2v15     w2v16     w2v17     w2v18     w2v19  \
business_id                                                                
--I7YYLada0tSLkORTHb5Q  0.016274  0.011641 -0.006097  0.007114 -0.016152   
--U98MNlDym2cLn36BBPgQ  0.003445  0.014191  0.000926  0.001358 -0.006260   
--j-kaNMCo1-DYzddCsA5Q  0.011731  0.034843  0.023824  0.004841 -0.003642   
--wIGbLEhlpl_UeAIyDmZQ  0.015487  0.007607  0.013315 -0.011175 -0.009165   
-000aQFeK6tqVLndf7xORg  0.021661  0.007791  0.023143 -0.019433 -0.010503   

                           w2v20     w2v21     w2v22     w2v23     w2v24  \
business_id                                                                
--I7YYLada0tSLkORTHb5Q -0.021916 -0.002982  0.006849 -0.020937 -0.001696   
--U98MNlDym2cLn36BBPgQ -0.016990  0.007976 -0.001236 -0.029893 -0.004399   
--j-kaNMCo1-DYzddCsA5Q  0.033550  0.023334  0.040423 -0.025388 -0.005392   
--wIGbLEhlpl_UeAIyDmZQ  0.002724 -0.008716 -0.013071 -0.023400  0.003305   
-000aQFeK6tqVLndf7xORg  0.016465  0.003757 -0.001521 -0.019992  0.001518   

                           w2v25     w2v26     w2v27     w2v28     w2v29  \
business_id                                                                
--I7YYLada0tSLkORTHb5Q  0.013037 -0.000854 -0.001098  0.000506 -0.009859   
--U98MNlDym2cLn36BBPgQ  0.022837 -0.005918  0.008592  0.012597 -0.005725   
--j-kaNMCo1-DYzddCsA5Q -0.012638  0.007508 -0.019039 -0.013676  0.005799   
--wIGbLEhlpl_UeAIyDmZQ  0.020969 -0.002856 -0.001024 -0.011011  0.005339   
-000aQFeK6tqVLndf7xORg  0.018253 -0.008955 -0.009514 -0.018659  0.013087   

                           w2v30     w2v31     w2v32     w2v33     w2v34  \
business_id                                                                
--I7YYLada0tSLkORTHb5Q -0.008329 -0.009132 -0.000561  0.030923 -0.028243   
--U98MNlDym2cLn36BBPgQ  0.009391  0.000858  0.003131  0.034826 -0.026948   
--j-kaNMCo1-DYzddCsA5Q -0.002171 -0.031701  0.005618  0.007776 -0.012970   
--wIGbLEhlpl_UeAIyDmZQ -0.000026 -0.009941 -0.001624 -0.009941  0.004110   
-000aQFeK6tqVLndf7xORg -0.008339 -0.016071  0.002549 -0.015986  0.011783   

                           w2v35     w2v36     w2v37     w2v38     w2v39  \
business_id                                                                
--I7YYLada0tSLkORTHb5Q  0.010541 -0.011148  0.005102 -0.010144  0.020217   
--U98MNlDym2cLn36BBPgQ -0.004410 -0.012987 -0.005924 -0.012023  0.018

In [21]:
all_features_business.describe()

w2v0          w2v1          w2v2          w2v3          w2v4  \
count  13942.000000  13942.000000  13942.000000  13942.000000  13942.000000   
mean       0.007080      0.016276     -0.008610      0.013150     -0.003761   
std        0.015540      0.012944      0.014396      0.024892      0.018633   
min       -0.103618     -0.045055     -0.075968     -0.127295     -0.112152   
25%       -0.003049      0.008878     -0.017373     -0.001154     -0.014281   
50%        0.008021      0.016514     -0.010473      0.014938     -0.002578   
75%        0.017971      0.023925     -0.001195      0.028679      0.008159   
max        0.081535      0.100700      0.068078      0.169776      0.082428   

               w2v5          w2v6          w2v7          w2v8          w2v9  \
count  13942.000000  13942.000000  13942.000000  13942.000000  13942.000000   
mean       0.006367      0.001918     -0.015486     -0.008427     -0.006332   
std        0.008587      0.014418      0.012549      0.014241      0.014275   
min       -0.042858     -0.056598     -0.092377     -0.095111     -0.072498   
25%        0.001081     -0.008472     -0.023362     -0.017819     -0.014199   
50%        0.006240      0.002227     -0.015916     -0.009443     -0.006280   
75%        0.011726      0.012906     -0.007534      0.000479      0.001247   
max        0.064519      0.070305      0.044968      0.062329      0.114934   

              w2v10         w2v11         w2v12         w2v13         w2v14  \
count  13942.000000  13942.000000  13942.000000  13942.000000  13942.000000   
mean      -0.001224     -0.009088      0.006823     -0.003458     -0.001781   
std        0.011323      0.018902      0.016522      0.014457      0.011241   
min       -0.092387     -0.087607     -0.082485     -0.072333     -0.063792   
25%       -0.006570     -0.020912     -0.004226     -0.013518     -0.008543   
50%       -0.000930     -0.010580      0.006891     -0.003530     -0.002140   
75%        0.004878      0.000685      0.018849      0.006920      0.004444   
max        0.054392      0.156033      0.079815      0.071259      0.071192   

              w2v15         w2v16         w2v17         w2v18         w2v19  \
count  13942.000000  13942.000000  13942.000000  13942.000000  13942.000000   
mean       0.009850      0.012324      0.002345     -0.001627     -0.010833   
std        0.012871      0.012133      0.014223      0.014394      0.012505   
min       -0.064000     -0.053064     -0.078378     -0.097343     -0.136704   
25%        0.002766      0.005113     -0.006937     -0.011310     -0.015789   
50%        0.010038      0.012895      0.002814     -0.000573     -0.010121   
75%        0.017006      0.019683      0.011747      0.008672     -0.004263   
max        0.084164      0.083976      0.079643      0.069839      0.040289   

              w2v20         w2v21         w2v22         w2v23         w2v24  \
count  13942.000000  13942.000000  13942.000000  13942.000000  13942.000000   
mean      -0.007057     -0.003667      0.000156     -0.022620      0.003001   
std        0.021097      0.012976      0.013166      0.010789      0.010887   
min       -0.105200     -0.084593     -0.120736     -0.087313     -0.063458   
25%       -0.022003     -0.011228     -0.007945     -0.029230     -0.003556   
50%       -0.010503     -0.003701      0.000256     -0.022559      0.003070   
75%        0.008635      0.003749      0.008072     -0.016150      0.009774   
max        0.090283      0.066703      0.080682      0.036760      0.058032   

              w2v25         w2v26         w2v27         w2v28         w2v29  \
count  13942.000000  13942.000000  13942.000000  13942.000000  13942.000000   
mean       0.009150      0.000020      0.003259     -0.007920      0.001406   
std        0.012970      0.010908      0.015610      0.018832      0.012652   
min       -0.073761     -0.059174     -0.065158     -0.090041     -0.045856   
25%        0.001063     -0.006911     -0.005223     -0.01985

# Create Label y (Business categories)

In [22]:
def load_business_df(): 
    filename = r'../../data/business.json'
    new_list = []
    for line in open(filename):
       new_list.append(json.loads(line))
    return pd.DataFrame.from_records(new_list)

dfbusiness = load_business_df()

In [23]:
dfbusiness.head()

address  \
0     2818 E Camino Acequia Drive   
1            30 Eglinton Avenue W   
2       10110 Johnston Rd, Ste 15   
3   15655 W Roosevelt St, Ste 237   
4  4209 Stuart Andrew Blvd, Ste F   

                                          attributes             business_id  \
0                           {'GoodForKids': 'False'}  1SWheh84yJXfytovILXOAQ   
1  {'RestaurantsReservations': 'True', 'GoodForMe...  QXAEGFB4oINsVuTFxEYKFQ   
2  {'GoodForKids': 'True', 'NoiseLevel': 'u'avera...  gnKjwL_1w79qoiV3IC_xQQ   
3                                               None  xvX2CttrVhyG2z1dFg_0xw   
4  {'BusinessAcceptsBitcoin': 'False', 'ByAppoint...  HhyxOkGAM07SRYtlQ4wMFQ   

                                          categories         city  \
0                                  Golf, Active Life      Phoenix   
1  Specialty Food, Restaurants, Dim Sum, Imported...  Mississauga   
2                  Sushi Bars, Restaurants, Japanese    Charlotte   
3                      Insurance, Financial Services     Goodyear   
4  Plumbing, Shopping, Local Services, Home Servi...    Charlotte   

                                               hours  is_open   latitude  \
0                                               None        0  33.522143   
1  {'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...        1  43.605499   
2  {'Monday': '17:30-21:30', 'Wednesday': '17:30-...        1  35.092564   
3  {'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...        1  33.455613   
4  {'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...        1  35.190012   

    longitude                             name postal_code  review_count  \
0 -112.018481       Arizona Biltmore Golf Club       85016             5   
1  -79.652289       Emerald Chinese Restaurant     L5R 3E7           128   
2  -80.859132      Musashi Japanese Restaurant       28210           170   
3 -112.395596  Farmers Insurance - Paul Lorenz       85338             3   
4  -80.887223              Queen City Plumbing       28217             4   

   stars state  
0    3.0    AZ  
1    2.5    ON  
2    4.0    NC  
3    5.0    AZ  
4    4.0    NC

# Join x,y (feature matrix, category) using business_id

In [24]:
dfbusiness.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')

In [25]:
len(dfbusiness['stars'].unique())

9

In [26]:
# Add business details to features df
keep_cols = ['business_id', 'categories', 'review_count']
all_features_business = all_features_business.merge(dfbusiness[keep_cols], how='left', on='business_id') 

In [27]:
all_features_business.head()

business_id      w2v0      w2v1      w2v2      w2v3      w2v4  \
0  --I7YYLada0tSLkORTHb5Q  0.016423  0.028373 -0.017573  0.015987 -0.000105   
1  --U98MNlDym2cLn36BBPgQ  0.009526  0.020788 -0.013227  0.037498  0.006747   
2  --j-kaNMCo1-DYzddCsA5Q  0.035586  0.022514 -0.006759 -0.034605 -0.004300   
3  --wIGbLEhlpl_UeAIyDmZQ -0.006451  0.005224 -0.009912  0.012220 -0.001507   
4  -000aQFeK6tqVLndf7xORg -0.007433 -0.000011 -0.009821  0.013711 -0.007697   

       w2v5      w2v6      w2v7      w2v8      w2v9     w2v10     w2v11  \
0 -0.000613  0.022004 -0.017549 -0.007508 -0.007643 -0.004137  0.005752   
1 -0.004874  0.012273 -0.024117  0.008519 -0.009936 -0.003010 -0.007703   
2 -0.001136  0.001259 -0.030018 -0.024447 -0.019531  0.013469 -0.015355   
3  0.014311 -0.004426 -0.013908 -0.003054 -0.003489 -0.005158 -0.013398   
4  0.009369 -0.000267 -0.014385  0.013446 -0.006545 -0.004254 -0.008840   

      w2v12     w2v13     w2v14     w2v15     w2v16     w2v17     w2v18  \
0  0.021792  0.008580 -0.005153  0.016274  0.011641 -0.006097  0.007114   
1  0.018541  0.022424 -0.008550  0.003445  0.014191  0.000926  0.001358   
2  0.014754 -0.012143  0.011463  0.011731  0.034843  0.023824  0.004841   
3 -0.009127 -0.013979 -0.009750  0.015487  0.007607  0.013315 -0.011175   
4  0.006065 -0.020545 -0.014723  0.021661  0.007791  0.023143 -0.019433   

      w2v19     w2v20     w2v21     w2v22     w2v23     w2v24     w2v25  \
0 -0.016152 -0.021916 -0.002982  0.006849 -0.020937 -0.001696  0.013037   
1 -0.006260 -0.016990  0.007976 -0.001236 -0.029893 -0.004399  0.022837   
2 -0.003642  0.033550  0.023334  0.040423 -0.025388 -0.005392 -0.012638   
3 -0.009165  0.002724 -0.008716 -0.013071 -0.023400  0.003305  0.020969   
4 -0.010503  0.016465  0.003757 -0.001521 -0.019992  0.001518  0.018253   

      w2v26     w2v27     w2v28     w2v29     w2v30     w2v31     w2v32  \
0 -0.000854 -0.001098  0.000506 -0.009859 -0.008329 -0.009132 -0.000561   
1 -0.005918  0.008592  0.012597 -0.005725  0.009391  0.000858  0.003131   
2  0.007508 -0.019039 -0.013676  0.005799 -0.002171 -0.031701  0.005618   
3 -0.002856 -0.001024 -0.011011  0.005339 -0.000026 -0.009941 -0.001624   
4 -0.008955 -0.009514 -0.018659  0.013087 -0.008339 -0.016071  0.002549   

      w2v33     w2v34     w2v35     w2v36     w2v37     w2v38     w2v39  \
0  0.030923 -0.028243  0.010541 -0.011148  0.005102 -0.010144  0.020217   
1  0.034826 -0.026948 -0.004410 -0.012987 -0.005924 -0.012023  0.018018   
2  0.007776 -0.012970  0.015142 -0.005067  0.002006 -0.001829  0.037603   
3 -0.009941  0.004110  0.010796  0.001066 -0.000427  0.006601  0.025162   
4 -0.015986  0.011783  0.018200  0.001336 -0.005553 -0.003634  0.033946   

      w2v40     w2v41     w2v42     w2v43     w2v44     w2v45     w2v46  \
0 -0.001866  0.043708 -0.007104 -0.005223 -0.006323 -0.014395 -0.018785   
1  0.005187  0.033649 -0.007751 -0.013777  0.004336 -0.018224 -0.030893   
2  0.003339  0.036467 -0.016766  0.020584  0.008400  0.001669 -0.013346   
3  0.008402  0.005168  0.008002 -0.001039  0.015670  0.017713  0.001480   
4  0.020831  0.002149  0.002212 -0.004911  0.015449  0.023007  0.004104   

      w2v47     w2v48     w2v49     w2v50     w2v51     w2v52     w2v53  \
0 -0.008720  0.000136  0.008391  0.001354  0.005404  0.009978 -0.003280   
1 -0.015549  0.008527  0.005289 -0.000202 -0.001845  0.011829 -0.001018   
2 -0.013458 -0.011666 -0.016915  0.003241  0.013855  0.011129 -0.030245   
3 -0.007099 -0.009715  0.002707 -0.005726  0.007443 -0.017098 -0.014837   
4 -0.006299 -0.017385 -0.009145 -0.008275  0.019660 -0.010135 -0.007654   

      w2v54     w2v55     w2v56     w2v57     w2v58     w2v59     w2v60  \
0  0.001135  0.006917  0.009708  0.002898 -0.005693 -0.006635  0.008571   
1  0.005172  0.001364 -0.001056 -0.007330 -0.018776 -0.007521  0.000760   
2  0.008177 -0.018768  0.018454  0.008811 -0.000059 -0.003335 -0.006106   
3 -0.004003 -0.004510  0.007199 -0.000072  0.003028  0.004661 -0.001873   
4  0.00

In [28]:
all_features_business['categories'][0]

'Nightlife, Sports Bars, Restaurants, Bars, American (Traditional)'

In [29]:
all_features_business.head()

business_id      w2v0      w2v1      w2v2      w2v3      w2v4  \
0  --I7YYLada0tSLkORTHb5Q  0.016423  0.028373 -0.017573  0.015987 -0.000105   
1  --U98MNlDym2cLn36BBPgQ  0.009526  0.020788 -0.013227  0.037498  0.006747   
2  --j-kaNMCo1-DYzddCsA5Q  0.035586  0.022514 -0.006759 -0.034605 -0.004300   
3  --wIGbLEhlpl_UeAIyDmZQ -0.006451  0.005224 -0.009912  0.012220 -0.001507   
4  -000aQFeK6tqVLndf7xORg -0.007433 -0.000011 -0.009821  0.013711 -0.007697   

       w2v5      w2v6      w2v7      w2v8      w2v9     w2v10     w2v11  \
0 -0.000613  0.022004 -0.017549 -0.007508 -0.007643 -0.004137  0.005752   
1 -0.004874  0.012273 -0.024117  0.008519 -0.009936 -0.003010 -0.007703   
2 -0.001136  0.001259 -0.030018 -0.024447 -0.019531  0.013469 -0.015355   
3  0.014311 -0.004426 -0.013908 -0.003054 -0.003489 -0.005158 -0.013398   
4  0.009369 -0.000267 -0.014385  0.013446 -0.006545 -0.004254 -0.008840   

      w2v12     w2v13     w2v14     w2v15     w2v16     w2v17     w2v18  \
0  0.021792  0.008580 -0.005153  0.016274  0.011641 -0.006097  0.007114   
1  0.018541  0.022424 -0.008550  0.003445  0.014191  0.000926  0.001358   
2  0.014754 -0.012143  0.011463  0.011731  0.034843  0.023824  0.004841   
3 -0.009127 -0.013979 -0.009750  0.015487  0.007607  0.013315 -0.011175   
4  0.006065 -0.020545 -0.014723  0.021661  0.007791  0.023143 -0.019433   

      w2v19     w2v20     w2v21     w2v22     w2v23     w2v24     w2v25  \
0 -0.016152 -0.021916 -0.002982  0.006849 -0.020937 -0.001696  0.013037   
1 -0.006260 -0.016990  0.007976 -0.001236 -0.029893 -0.004399  0.022837   
2 -0.003642  0.033550  0.023334  0.040423 -0.025388 -0.005392 -0.012638   
3 -0.009165  0.002724 -0.008716 -0.013071 -0.023400  0.003305  0.020969   
4 -0.010503  0.016465  0.003757 -0.001521 -0.019992  0.001518  0.018253   

      w2v26     w2v27     w2v28     w2v29     w2v30     w2v31     w2v32  \
0 -0.000854 -0.001098  0.000506 -0.009859 -0.008329 -0.009132 -0.000561   
1 -0.005918  0.008592  0.012597 -0.005725  0.009391  0.000858  0.003131   
2  0.007508 -0.019039 -0.013676  0.005799 -0.002171 -0.031701  0.005618   
3 -0.002856 -0.001024 -0.011011  0.005339 -0.000026 -0.009941 -0.001624   
4 -0.008955 -0.009514 -0.018659  0.013087 -0.008339 -0.016071  0.002549   

      w2v33     w2v34     w2v35     w2v36     w2v37     w2v38     w2v39  \
0  0.030923 -0.028243  0.010541 -0.011148  0.005102 -0.010144  0.020217   
1  0.034826 -0.026948 -0.004410 -0.012987 -0.005924 -0.012023  0.018018   
2  0.007776 -0.012970  0.015142 -0.005067  0.002006 -0.001829  0.037603   
3 -0.009941  0.004110  0.010796  0.001066 -0.000427  0.006601  0.025162   
4 -0.015986  0.011783  0.018200  0.001336 -0.005553 -0.003634  0.033946   

      w2v40     w2v41     w2v42     w2v43     w2v44     w2v45     w2v46  \
0 -0.001866  0.043708 -0.007104 -0.005223 -0.006323 -0.014395 -0.018785   
1  0.005187  0.033649 -0.007751 -0.013777  0.004336 -0.018224 -0.030893   
2  0.003339  0.036467 -0.016766  0.020584  0.008400  0.001669 -0.013346   
3  0.008402  0.005168  0.008002 -0.001039  0.015670  0.017713  0.001480   
4  0.020831  0.002149  0.002212 -0.004911  0.015449  0.023007  0.004104   

      w2v47     w2v48     w2v49     w2v50     w2v51     w2v52     w2v53  \
0 -0.008720  0.000136  0.008391  0.001354  0.005404  0.009978 -0.003280   
1 -0.015549  0.008527  0.005289 -0.000202 -0.001845  0.011829 -0.001018   
2 -0.013458 -0.011666 -0.016915  0.003241  0.013855  0.011129 -0.030245   
3 -0.007099 -0.009715  0.002707 -0.005726  0.007443 -0.017098 -0.014837   
4 -0.006299 -0.017385 -0.009145 -0.008275  0.019660 -0.010135 -0.007654   

      w2v54     w2v55     w2v56     w2v57     w2v58     w2v59     w2v60  \
0  0.001135  0.006917  0.009708  0.002898 -0.005693 -0.006635  0.008571   
1  0.005172  0.001364 -0.001056 -0.007330 -0.018776 -0.007521  0.000760   
2  0.008177 -0.018768  0.018454  0.008811 -0.000059 -0.003335 -0.006106   
3 -0.004003 -0.004510  0.007199 -0.000072  0.003028  0.004661 -0.001873   
4  0.00

In [30]:
def stringDFColToBinaryCols(df, series_name):
    # Create list of all categories
    all_cats = []
    for string in df[series_name]:
        string = str(string)
        cats = string.strip().replace(' ', '').split(',')
        for cat in cats:
            if cat not in all_cats:
                all_cats.append(cat)
    # Make binary for each cat for each row
    for cat in all_cats:
        df[cat] = df[series_name].str.strip().str.replace(' ', '').str.contains(cat)
        # This technique will have some problems. 'Golf' may appear in non-Golf categories (ie 'Disc Golf')
        # Can be fixed with regular expressions: ',Golf,' OR 'BOF Golf,' OR ',Golf EOF'
    
    return df, all_cats
        
all_features_business, all_cats = stringDFColToBinaryCols(all_features_business, 'categories')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


In [31]:
print(all_cats)

['Nightlife', 'SportsBars', 'Restaurants', 'Bars', 'American(Traditional)', 'Pizza', 'HairRemoval', 'NailTechnicians', 'Beauty&Spas', 'NailSalons', 'Waxing', 'DaySpas', 'Electronics', 'ProfessionalServices', 'LocalServices', 'ElectronicsRepair', 'Computers', 'Shopping', 'Automotive', 'AutoRepair', 'Chinese', 'EyelashService', 'TobaccoShops', 'VapeShops', 'CarDealers', 'UsedCarDealers', 'Dentists', 'GeneralDentistry', 'CosmeticDentists', 'PediatricDentists', 'Health&Medical', 'Tex-Mex', 'Mexican', 'Arts&Entertainment', 'Festivals', 'Food', 'FoodTrucks', 'FarmersMarket', 'Portuguese', 'Bakeries', 'ChickenShop', 'Barbeque', 'EventPlanning&Services', 'EventPhotography', 'Photographers', 'SessionPhotography', 'SkinCare', 'Antiques', 'IceCream&FrozenYogurt', 'Donuts', 'SpecialtyFood', 'WebDesign', 'GraphicDesign', 'Marketing', 'RecyclingCenter', 'Caterers', 'Southern', 'ComfortFood', 'Breakfast&Brunch', 'French', 'American(New)', 'Burgers', 'Sandwiches', 'Coffee&Tea', 'Brasseries', 'Gyms', '

In [32]:
print(
    len(all_features_business[all_features_business['Golf']==True]), 
    len(all_features_business[all_features_business['DiscGolf']==True]), 
)

61 1


In [48]:
print(all_features_business[all_features_business['DiscGolf']==True]['categories'].values)
print('Should not have a True value for Golf, but does. Problem to deal with in the future.')
print(all_features_business[all_features_business['DiscGolf']==True]['Golf'].values)

['Sporting Goods, Active Life, Bike Rentals, Disc Golf, Shopping']
Should not have a True value for Golf, but does. Problem to deal with in the future.
[True]


In [35]:
all_features_business.head()

business_id      w2v0      w2v1      w2v2      w2v3      w2v4  \
0  --I7YYLada0tSLkORTHb5Q  0.016423  0.028373 -0.017573  0.015987 -0.000105   
1  --U98MNlDym2cLn36BBPgQ  0.009526  0.020788 -0.013227  0.037498  0.006747   
2  --j-kaNMCo1-DYzddCsA5Q  0.035586  0.022514 -0.006759 -0.034605 -0.004300   
3  --wIGbLEhlpl_UeAIyDmZQ -0.006451  0.005224 -0.009912  0.012220 -0.001507   
4  -000aQFeK6tqVLndf7xORg -0.007433 -0.000011 -0.009821  0.013711 -0.007697   

       w2v5      w2v6      w2v7      w2v8      w2v9     w2v10     w2v11  \
0 -0.000613  0.022004 -0.017549 -0.007508 -0.007643 -0.004137  0.005752   
1 -0.004874  0.012273 -0.024117  0.008519 -0.009936 -0.003010 -0.007703   
2 -0.001136  0.001259 -0.030018 -0.024447 -0.019531  0.013469 -0.015355   
3  0.014311 -0.004426 -0.013908 -0.003054 -0.003489 -0.005158 -0.013398   
4  0.009369 -0.000267 -0.014385  0.013446 -0.006545 -0.004254 -0.008840   

      w2v12     w2v13     w2v14     w2v15     w2v16     w2v17     w2v18  \
0  0.021792  0.008580 -0.005153  0.016274  0.011641 -0.006097  0.007114   
1  0.018541  0.022424 -0.008550  0.003445  0.014191  0.000926  0.001358   
2  0.014754 -0.012143  0.011463  0.011731  0.034843  0.023824  0.004841   
3 -0.009127 -0.013979 -0.009750  0.015487  0.007607  0.013315 -0.011175   
4  0.006065 -0.020545 -0.014723  0.021661  0.007791  0.023143 -0.019433   

      w2v19     w2v20     w2v21     w2v22     w2v23     w2v24     w2v25  \
0 -0.016152 -0.021916 -0.002982  0.006849 -0.020937 -0.001696  0.013037   
1 -0.006260 -0.016990  0.007976 -0.001236 -0.029893 -0.004399  0.022837   
2 -0.003642  0.033550  0.023334  0.040423 -0.025388 -0.005392 -0.012638   
3 -0.009165  0.002724 -0.008716 -0.013071 -0.023400  0.003305  0.020969   
4 -0.010503  0.016465  0.003757 -0.001521 -0.019992  0.001518  0.018253   

      w2v26     w2v27     w2v28     w2v29     w2v30     w2v31     w2v32  \
0 -0.000854 -0.001098  0.000506 -0.009859 -0.008329 -0.009132 -0.000561   
1 -0.005918  0.008592  0.012597 -0.005725  0.009391  0.000858  0.003131   
2  0.007508 -0.019039 -0.013676  0.005799 -0.002171 -0.031701  0.005618   
3 -0.002856 -0.001024 -0.011011  0.005339 -0.000026 -0.009941 -0.001624   
4 -0.008955 -0.009514 -0.018659  0.013087 -0.008339 -0.016071  0.002549   

      w2v33     w2v34     w2v35     w2v36     w2v37     w2v38     w2v39  \
0  0.030923 -0.028243  0.010541 -0.011148  0.005102 -0.010144  0.020217   
1  0.034826 -0.026948 -0.004410 -0.012987 -0.005924 -0.012023  0.018018   
2  0.007776 -0.012970  0.015142 -0.005067  0.002006 -0.001829  0.037603   
3 -0.009941  0.004110  0.010796  0.001066 -0.000427  0.006601  0.025162   
4 -0.015986  0.011783  0.018200  0.001336 -0.005553 -0.003634  0.033946   

      w2v40     w2v41     w2v42     w2v43     w2v44     w2v45     w2v46  \
0 -0.001866  0.043708 -0.007104 -0.005223 -0.006323 -0.014395 -0.018785   
1  0.005187  0.033649 -0.007751 -0.013777  0.004336 -0.018224 -0.030893   
2  0.003339  0.036467 -0.016766  0.020584  0.008400  0.001669 -0.013346   
3  0.008402  0.005168  0.008002 -0.001039  0.015670  0.017713  0.001480   
4  0.020831  0.002149  0.002212 -0.004911  0.015449  0.023007  0.004104   

      w2v47     w2v48     w2v49     w2v50     w2v51     w2v52     w2v53  \
0 -0.008720  0.000136  0.008391  0.001354  0.005404  0.009978 -0.003280   
1 -0.015549  0.008527  0.005289 -0.000202 -0.001845  0.011829 -0.001018   
2 -0.013458 -0.011666 -0.016915  0.003241  0.013855  0.011129 -0.030245   
3 -0.007099 -0.009715  0.002707 -0.005726  0.007443 -0.017098 -0.014837   
4 -0.006299 -0.017385 -0.009145 -0.008275  0.019660 -0.010135 -0.007654   

      w2v54     w2v55     w2v56     w2v57     w2v58     w2v59     w2v60  \
0  0.001135  0.006917  0.009708  0.002898 -0.005693 -0.006635  0.008571   
1  0.005172  0.001364 -0.001056 -0.007330 -0.018776 -0.007521  0.000760   
2  0.008177 -0.018768  0.018454  0.008811 -0.000059 -0.003335 -0.006106   
3 -0.004003 -0.004510  0.007199 -0.000072  0.003028  0.004661 -0.001873   
4  0.00

In [49]:
# Clean

# Remove rows with NaNs
print('Before: ', len(all_features_business))
all_features_business = all_features_business.dropna(axis=0)
print('After:  ', len(all_features_business))

Before:  13922
After:   13922


In [50]:
# Create final y and x 

y_df = all_features_business[all_cats]
x_cols = [ele for ele in all_features_business.columns if ele not in all_cats+['categories', 'business_id']]
# May also want to remove from x_cols: 'cool', 'funny', 'useful', 'stars', 'categories', 'review_count' 

x_df = all_features_business[x_cols]

x = x_df.values
y = y_df.values

# Classifier wants 1/0, not T/F
y = y.astype(int)

In [51]:
y

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [82]:
# Train/Test Split

from sklearn.model_selection import train_test_split

X_train, X_trest, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Category Prediction

In [83]:
# Multilabel Classification
# RandomForestClassifier supports multilabel classification

# Most other classifiers will require use of 
    # sklearn.multioutput.MultiOutputClassifier to run a separate classifier model for each targe
    
from sklearn.ensemble import RandomForestClassifier

In [84]:
rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1)

In [85]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Plot ROC curve to assess

In our case we want a TPR (True Positive Rate, plotted on the y-axis) close to 1 since we want to Recall ALL correct categories. 

We also want a FPR (False Positive Rate, plotted on the x-axis) that is nonzero, but probably the smallest value that maximizes TPR. This is because we want some FPs (False Positives) because they are WHAT WE ARE RECOMMENDING. 

In [94]:
from sklearn.metrics import plot_roc_curve

ax = plt.gca()
rfc_disp = plot_roc_curve(rfc, X_test, y_test, ax=ax)
plt.show()

ImportError: cannot import name 'plot_roc_curve' from 'sklearn.metrics' (/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/__init__.py)

In [91]:
break

SyntaxError: 'break' outside loop (<ipython-input-91-6aaf1f276005>, line 4)

In [56]:
predict0 = rfc.predict(x[0].reshape(1,-1))
predict0

array([[1., 1., 1., ..., 0., 0., 0.]])

In [71]:
all_cats_ser = pd.Series(data=all_cats)
y0cats = all_cats_ser.loc[y[0]==1]
list(y0cats)

['Nightlife', 'SportsBars', 'Restaurants', 'Bars']

In [72]:
all_cats_ser.loc[range(5)]

0                Nightlife
1               SportsBars
2              Restaurants
3                     Bars
4    American(Traditional)
dtype: object

In [ ]:
all_features_business.iloc[0]

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(range(predict0.shape[1]), predict0[0])

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(range(predict0.shape[1]), y[0])

In [ ]:
break

In [ ]:
# Cluster users using K-means
from sklearn.cluster import MiniBatchKMeans

max_clusters = 200 # 10
kmeans_cost = []
for num_clusters in range(1,max_clusters):
    k_means_clutering = MiniBatchKMeans(n_clusters=num_clusters)
    k_means_clutering.fit(reviewDataVecs[~np.isnan(reviewDataVecs).any(axis=1)]) # Drop rows that have NAN
    kmeans_cost.append(k_means_clutering.inertia_)
    

In [ ]:
# Determine the best value of K to use (the number of clusters)
# plot the cost against K values 
plt.plot(range(1, max_clusters), kmeans_cost, color ='g', linewidth ='3') 
plt.title('20 Clusters of Reviews:', fontsize=20)
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
plt.show() # clear the plot 

In [ ]:
# Cluster users using K-means
# Interpret user clusters

from sklearn.cluster import KMeans

num_clusters = 20
k_means_clutering = KMeans(n_clusters=num_clusters)
k_means_clutering.fit(reviewDataVecs[~np.isnan(reviewDataVecs).any(axis=1)])

In [ ]:
cluster_centers_df = pd.DataFrame(data=k_means_clutering.cluster_centers_, columns=all_features_df.iloc[:,:-4].columns)
cluster_centers_df.head()

In [ ]:
cluster_centers_df = cluster_centers_df.merge(right=all_features_df.dropna()[['cool', 'funny', 'useful', 'stars']].groupby(by=k_means_clutering.labels_).mean(), right_index=True, left_index=True)


In [ ]:
cluster_centers_df.head()

In [ ]:
cluster_centers_df.iloc[0]['stars']

In [ ]:
from wordcloud import WordCloud

for cluster in range(len(cluster_centers_df)):
#     series = user_cluster_centers_df.iloc[cluster,:-4] #Use relevant row, drop non-word cols
#     wc_dict = series.to_dict()
    series = model.wv.similar_by_vector(cluster_centers_df.iloc[cluster,:-4].values, topn=100)
    wc_dict = {}
    for key, val in series:
        wc_dict[key] = val
    wordcloud = WordCloud().generate_from_frequencies(wc_dict)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title('Cluster{0:3d}: {1:.2f}stars, {2:.2f}useful'.format(cluster, cluster_centers_df.iloc[cluster]['stars'], 
                                                    cluster_centers_df.iloc[cluster]['useful']), fontsize=20)
    plt.axis('off')
    plt.show()
    

In [ ]:
print('The most similar words to the avg vector describing each user cluster:\n')
for cluster in range(len(cluster_centers_df)):
    print('Cluster {}'.format(cluster))
    display(
        model.wv.similar_by_vector(cluster_centers_df.iloc[cluster,:].values, topn=10)
    )

# Cluster with metadata (useful, cool, funny, stars)

In [ ]:
# Cluster users using K-means
from sklearn.cluster import MiniBatchKMeans

max_clusters = 200 # 10
kmeans_cost = []
for num_clusters in range(1,max_clusters):
    k_means_clutering = MiniBatchKMeans(n_clusters=num_clusters)
    k_means_clutering.fit(all_features_df.dropna()) # Drop rows that have NAN
    kmeans_cost.append(k_means_clutering.inertia_)
    

In [ ]:
# Determine the best value of K to use (the number of clusters)
# plot the cost against K values 
plt.plot(range(1, max_clusters), kmeans_cost, color ='g', linewidth ='3') 
plt.title('20 Clusters of Reviews:', fontsize=20)
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
plt.show() # clear the plot 

In [ ]:
# Cluster users using K-means
# Interpret user clusters

from sklearn.cluster import KMeans

num_clusters = 20
k_means_clutering = KMeans(n_clusters=num_clusters)
k_means_clutering.fit(all_features_df.dropna())

In [ ]:
cluster_centers_df = pd.DataFrame(data=k_means_clutering.cluster_centers_, columns=all_features_df.iloc[:,:].columns)
# cluster_centers_df = cluster_centers_df.merge(right=all_features_df.dropna()[['cool', 'funny', 'useful', 'stars']].groupby(by=k_means_clutering.labels_).mean(), right_index=True, left_index=True)
cluster_centers_df.head()


In [ ]:
from wordcloud import WordCloud

for cluster in range(len(cluster_centers_df)):
#     series = user_cluster_centers_df.iloc[cluster,:-4] #Use relevant row, drop non-word cols
#     wc_dict = series.to_dict()
    series = model.wv.similar_by_vector(cluster_centers_df.iloc[cluster,:-4].values, topn=100)
    wc_dict = {}
    for key, val in series:
        wc_dict[key] = val
    wordcloud = WordCloud().generate_from_frequencies(wc_dict)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title('Cluster{0:3d}: {1:.2f}stars, {2:.2f}useful'.format(cluster, cluster_centers_df.iloc[cluster]['stars'], 
                                                    cluster_centers_df.iloc[cluster]['useful']), fontsize=20)
    plt.axis('off')
    plt.show()
    

In [ ]:
for col in ['cool', 'funny', 'useful', 'stars']:
    plt.bar(x=range(len(cluster_centers_df)), height=cluster_centers_df[col])
    plt.title('{}'.format(col))
    plt.show()
# display(cluster_centers_df[['cool', 'funny', 'useful', 'stars']])